Challenge IA

In [206]:
import base64
from mistralai import Mistral

api_key = "Yp0Uo7Vx4uSJIlc94dj3MA5ME71KpwIR"


Code pour format PDF

In [207]:
#Pour recupérer fichier PDF uploadé, il faut ensuite récuperer l url lié au fichier

uploaded_pdf = client.files.upload(
    file={
        "file_name": "cv.pdf",
        "content": open("joel.pdf", "rb"),
    },
    purpose="ocr"
)  

#on recupère l url
pdf_url = client.files.get_signed_url(file_id=uploaded_pdf.id).url  # Récupère l'URL proprement

In [208]:
#On recupère l url du pdf et on extrait les informations 
ocr_response = client.ocr.process(
    model="mistral-ocr-latest",
    document={
        "type": "document_url",
        "document_url": pdf_url
    },
    include_image_base64=True
)

In [221]:
#Reponse du llm avec les infos extraites du CV pdf 
ocr_text = ocr_response.pages[0].markdown

In [ ]:
# #Code pour convertir PDF en image si besoin?

# import fitz  # PyMuPDF
# from PIL import Image

# # Chemin du fichier PDF
# pdf_path = "hugo.pdf"

# # Ouvrir le PDF
# doc = fitz.open(pdf_path)

# # Convertir chaque page en image
# for page_num in range(len(doc)):
#     page = doc[page_num]
#     pix = page.get_pixmap()  # Convertir en image
#     img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    
#     # Sauvegarde sous forme de fichier image
#     img.save(f"page_{page_num + 1}.png", "PNG")

# print("Conversion terminée : les images sont enregistrées.")



Conversion terminée : les images sont enregistrées.


Code pour format IMAGE

In [ ]:
#Code pour encoder une image, qui sera ensuite donné au LLM 
def encode_image(image_path):
    """Encode the image to base64."""
    try:
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    except FileNotFoundError:
        print(f"Error: The file {image_path} was not found.")
        return None
    except Exception as e:  # Added general exception handling
        print(f"Error: {e}")
        return None

# Path to your image
image_path = "page_1.png"

# Getting the base64 string
base64_image = encode_image(image_path)

ocr_response = client.ocr.process(
    model="mistral-ocr-latest",
    document={
        "type": "image_url",
        "image_url": f"data:image/jpeg;base64,{base64_image}" 
    }
)

In [222]:
#print(ocr_response)

ocr_text = ocr_response.pages[0].markdown  # Texte brut extrait de l'image
print(ocr_text)

# Joël SOLLARI 

## De reconversion, je recherche un stage de 4 à 6 mois sur un poste de data scientist à partir du 17/03/2025.

ÉDUCATION
Master 2 Statistique et
Informatique pour la Science
des donnéEs (SISE)
Université Lyon 2 (2025)
Maitrise Informatique
Aix-Marseille Université (2021)

Licence d'informatique
CTES - Aix-Marseille
Université (2020)

## COMPETENCES

Avancées :
Python, Machine Learning, NLP, SQL, ETL / ELT,

## Intermédiaires :

NoSQL, Deep Learning, MLOps, Conception des entrepôts de données, Statistiques, Git

## Complémentaires :

R, LLM, Développement d'API, Power BI, Tableau, Qlik, Microsoft Azure, Séries Temporelles, Dash, Shiny

## A ng l a is

Avancé (B2-C1)

## EXPÉRIENCES

Depuis 2017
Support technique, informatique et méthodologique (Ingénieur de recherche) - Préparation de corpus - Gestion bases de données $\cdot$ Amalteya SAS

Publications :

- Finding the best trade-off between performance and interpretability in predicting hospital length of stay using s

In [219]:
ocr_text

Pour traiter les données extraites depuis l'OCR mistral

Avec un LLM

In [223]:
# Prompt pour le LLM
prompt = f"""
Je vais te donner un texte extrait d'un CV au format OCR. Ton objectif est d'extraire et structurer les informations sous forme de JSON. 

Voici le texte extrait :
{ocr_text}

### Format de sortie JSON souhaité :
{{
    "formations": [
        {{
            "nom": "[Nom de la formation]",
            "dates": "[Dates]",
            "institution": "[Institution]"
        }}
    ],
    "experiences_professionnelles": [
        {{
            "poste": "[Titre du poste]",
            "dates": "[Dates]",
            "entreprise": "[Entreprise]",
            "description": "[Brève description]"
        }}
    ],
    "competences": [
        "[Compétence 1]",
        "[Compétence 2]",
        "[Compétence 3]"
    ]
}}

Respecte ce format strict en JSON sans ajouter d'explications supplémentaires. 
Si certaines informations sont absentes, laisse les champs vides.
"""


# Envoyer le prompt au LLM
response = client.chat.complete(
    model="open-mistral-7b",
    messages=[
        {
            "role": "system","content": (
             "Tu êtes un assistant qui extrait des informations et les retourne au format demandé. Tu dois rester logique dans la réponse, donc si certaines informations semblent fausses, ne les renvoie pas."
            ),
        
        },
        {"role":"user","content": prompt,},
    ],
)

# Afficher la réponse
print(response.choices[0].message.content)

{
    "formations": [
        {
            "nom": "Master 2 Statistique et Informatique pour la Science des données (SISE)",
            "dates": "2025",
            "institution": "Université Lyon 2"
        },
        {
            "nom": "Maitrise Informatique",
            "dates": "2021",
            "institution": "Aix-Marseille Université"
        },
        {
            "nom": "Licence d'informatique",
            "dates": "2020",
            "institution": "CTES - Aix-Marseille Université"
        }
    ],
    "experiences_professionnelles": [
        {
            "poste": "Support technique, informatique et méthodologique (Ingénieur de recherche)",
            "dates": "Depuis 2017",
            "entreprise": "Préparation de corpus - Gestion bases de données $\cdot$ Amalteya SAS",
            "description": "Publications : - Finding the best trade-off between performance and interpretability in predicting hospital length of stay using structured and unstructured data (2023

In [224]:
# Prompt pour le LLM
prompt = f"""
Je vais te donner un texte extrait d'un CV au format OCR. Ton objectif est de me dire si sa candidature est intéressante pour une offre de stage de Data Scientist?

Voici le texte extrait :
{ocr_text}
"""


# Envoyer le prompt au LLM
response = client.chat.complete(
    model="open-mistral-7b",
    messages=[
        {
            "role": "system","content": (
             "Tu êtes un assistant qui extrait des informations et les retourne au format demandé. Tu dois rester logique dans la réponse, donc si certaines informations semblent fausses, ne les renvoie pas."
            ),
        
        },
        {"role":"user","content": prompt,},
    ],
)

# Afficher la réponse
print(response.choices[0].message.content)

En étudiant le CV, il est clair que Joël Sollari a une formation académique appropriée pour le poste de Data Scientist, avec un Master 2 en Statistique et Informatique pour la Science des données (SISE) à l'Université Lyon 2 et une Maîtrise en Informatique à l'Aix-Marseille Université. Il a également une expérience pratique en tant qu'Ingénieur de recherche dans un poste technique, où il a travaillé sur la préparation de corpus et la gestion de bases de données.

En examinant les compétences de Joël Sollari, il est apparent qu'il a des compétences avancées en Python, Machine Learning, NLP, SQL, ETL / ELT, ainsi qu'un niveau intermédiaire en NoSQL, Deep Learning, MLOps, Conception des entrepôts de données, Statistiques, Git. Il a également des compétences complémentaires en R, LLM, Développement d'API, Power BI, Tableau, Qlik, Microsoft Azure, Séries Temporelles, Dash, Shiny.

En tenant compte de ses compétences et de son expérience, il semble que Joël Sollari pourrait être un bon candi